In [24]:
import sys
import os
import logging
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError


sys.path.append(os.path.abspath('../'))
from src.params import Params
from src.client import DatabaseClient
from src.logging_config import setup_logging

In [25]:
setup_logging()

In [26]:
# Path to the Grammy's dataset
csv_file = '../data/external/the_grammy_awards.csv'

df = pd.read_csv(csv_file)

df.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


A Parameters class centralizes anything that can be parametrized in the code. As we want to use the parameters for the connection to the PostgrSQL databases, params are instantiated as `params = Params()`.

In [27]:
# Parames instance
params = Params()

# Connection to the database
db_client = DatabaseClient(params)

2025-03-29 16:03:58,821 - INFO - root - Successfully connected to the database.


A `table_name` for the grammy's dataset table is defined and then the Dataframe is writed to the PostgreSWL with SQLAlchemy's mehtod `df.to.sql()`

In [28]:
table_name = 'grammys_raw'

try:
    # Write the DataFrame to the database table
    df.to_sql(table_name, con=db_client.engine, if_exists='replace', index=False)
    logging.info(f"CSV data has been successfully loaded into the table '{table_name}'.")
except Exception as e:
    logging.error(f"Failed to load CSV data into the table '{table_name}'.")
    logging.error(f"Error details: {e}")


2025-03-29 16:03:59,074 - INFO - root - CSV data has been successfully loaded into the table 'grammys_raw'.


A `excute_queries()` function is defined to run a list of SQL queries to compare the dimensions of the Dataframa and the PostgreSQL table.

In [29]:
def execute_queries(queries, db_client):
    """
    Execute a list of SQL queries and log the results.
    
    Parameters:
    - queries: List of SQL query strings.
    - db_client: DatabaseClient object containing the connection engine.
    """
    try:
        with db_client.engine.connect() as connection:
            for query in queries:
                logging.info(f"Executing query: {query}")
                # Convert query string to executable SQLAlchemy text object
                executable_query = text(query)
                result = connection.execute(executable_query)
                rows = result.fetchall()
                
                # Print and log the query results
                logging.info("Query results:")
                for row in rows:
                    logging.info(row)
    except SQLAlchemyError as e:
        logging.error("Failed to execute queries.")
        logging.error(f"Error details: {e}")

Two queries are defined in a list to count the rows and columns of the "grammys_raw" table. Then they are executed through the `excute_queries()` function.

In [30]:
queries = [
    "SELECT COUNT(*) FROM grammys_raw",
    "SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'grammys_raw'"
]


execute_queries(queries, db_client)

2025-03-29 16:03:59,106 - INFO - root - Executing query: SELECT COUNT(*) FROM grammys_raw
2025-03-29 16:03:59,114 - INFO - root - Query results:
2025-03-29 16:03:59,116 - INFO - root - (4810,)
2025-03-29 16:03:59,117 - INFO - root - Executing query: SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'grammys_raw'
2025-03-29 16:03:59,127 - INFO - root - Query results:
2025-03-29 16:03:59,128 - INFO - root - (10,)


The output indicates the table has 4810 rows and 10 columns. While the method '.shape' used for the grammy's dataset Dataframe returns the same dimension, so the insertion is consistent.

In [31]:
df.shape

(4810, 10)

Finally, the connection to the DatabaseClient is closed through its method `.close()`.

In [ ]:
try:
    db_client.close()
except Exception as e:
    logging.error(f"Failed to close connection to the database.")
    logging.error(f"Error details: {e}")

2025-03-29 16:03:59,156 - INFO - root - Connection to database closed successfully.
